<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>CatBoost</a></span><ul class="toc-item"><li><span><a href="#аргумент-text_features" data-toc-modified-id="аргумент-text_features-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>аргумент text_features</a></span></li><li><span><a href="#Pipeline-с-tf-idf" data-toc-modified-id="Pipeline-с-tf-idf-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Pipeline с tf-idf</a></span></li></ul></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#KNeighborsClassifier" data-toc-modified-id="KNeighborsClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>KNeighborsClassifier</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>LogisticRegression</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span><ul class="toc-item"><li><span><a href="#Данные:" data-toc-modified-id="Данные:-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Данные:</a></span></li><li><span><a href="#Модели:" data-toc-modified-id="Модели:-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Модели:</a></span></li></ul></ul></div>

# Классификация токсичных комментариев для «Викишоп»

Цель проекта — обучить модель для автоматического определения токсичных пользовательских комментариев на платформе интернет-магазина «Викишоп». Это поможет направлять оскорбительные правки на модерацию.

Источник данных: размеченный набор toxic_comments.csv с текстами комментариев и бинарной меткой токсичности.

Основные этапы:
- Предобработка текста и токенизация.
- Обучение и сравнение нескольких моделей машинного обучения.
- Оценка качества моделей по метрике F1 (целевой порог — не ниже 0.75).

Проект демонстрирует навыки текстовой предобработки, работы с моделями классификации и оценки качества моделей.

## Подготовка

!pip install -q catboost

In [1]:
!pip install protobuf==3.20.*
!pip install spacy
!spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 217.9 kB/s eta 0:00:59
     --------------------------------------- 0.0/12.8 MB 326.8 kB/s eta 0:00:40
     --------------------------------------- 0.1/12.8 MB 655.4 kB/s eta 0:00:20
      --------------------------------------- 0.2/12.8 MB 1.1 MB/s eta 0:00:11
     - -------------------------------------- 0.3/12.8 MB 1.4 MB/s eta 0:00:09
     -- ------------------------------------- 0.7/12.8 MB 2.3 MB/s eta 0:00:06
     --- ------------------------------------ 1.0/12.8 MB 3.0 MB/s eta 0:00:04
     ---- ----------------------------------- 1.5/12.8 MB 4.0 MB/s eta 0:00:03
     ------ --------------------------------- 2.1/12.8 MB 4.9 MB/s eta 0:00:03
     -------- ------------------------------- 2.6/12.8 MB 5.6 MB/s eta 0:00:02
     --------- ------------------------------ 3.0/12.8 MB 5.9 MB/s eta 0:00:02
     --------- ------------------------------ 3.0/12.8 M

2023-04-16 17:34:23.265427: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-04-16 17:34:23.265787: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-16 17:34:29.604214: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-04-16 17:34:29.604560: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2023-04-16 17:34:29.604893: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2023-04-16 17:34:29.605220: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cu

In [11]:
import pandas as pd
import numpy as np
import re

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import f1_score

import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer

from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [1]:
# Загрузка данных, к сожалению, по политиике Яндекса, показать не могу
data = pd.read_csv('toxic_comments.csv')

NameError: name 'pd' is not defined

In [4]:
def get_info(data: pd.DataFrame):
    display(data.head())
    print('\n INFO\n')
    data.info()
    print('\n Дупликаты общие \n')
    display(data[data.duplicated()])
    print('\n Пропущенные данные в % \n')
    display(data.isna().mean()*100)
    display(data.describe(include='all'))

In [5]:
get_info(data)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0



 INFO

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB

 Дупликаты общие 



,Unnamed: 0,text,toxic



 Пропущенные данные в % 



Unnamed: 0    0.0
text          0.0
toxic         0.0
dtype: float64

,Unnamed: 0,text,toxic
count,159292.000000,159292,159292.000000
unique,NaN,159292,NaN
top,NaN,List of present day countries that were part o...,NaN
freq,NaN,1,NaN
mean,79725.697242,NaN,0.101612
std,46028.837471,NaN,0.302139
min,0.000000,NaN,0.000000
25%,39872.750000,NaN,0.000000
50%,79721.500000,NaN,0.000000
75%,119573.250000,NaN,0.000000


In [6]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [7]:
def clear_text(text):
    return " ".join((re.sub(r'[^a-zA-Z ]', ' ', text)).split())

In [9]:
def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

In [12]:
%%time
data['clear_text'] = data.text.apply(clear_text)
data['lemma_text'] = data.clear_text.apply(lemmatizer)

CPU times: total: 13min 27s
Wall time: 13min 27s


In [13]:
vectorizer = TfidfVectorizer(stop_words='english')

In [14]:
X = data['lemma_text']
Y = data['toxic']

In [15]:
features_train, features_test, targets_train, targets_test = train_test_split(X, Y,
                                                                              test_size=0.2)

## Обучение

### CatBoost

In [16]:
RANDOM_STATE=12345

In [ ]:
%%time
model_Cat = CatBoostClassifier(random_state=RANDOM_STATE, allow_writing_files=False, verbose=False, auto_class_weights='Balanced')


param_grid_Cat = {'catboost__learning_rate': [0.01, 0.05, 0.1, 0.3]}
#'depth': range(3, 10),
#'l2_leaf_reg': [1, 3, 5, 7, 9],}
tfidf_cat_pipe = Pipeline([
    ('tvec', vectorizer),
    ('catboost', model_Cat)
])

gs_Cat = GridSearchCV(
    tfidf_cat_pipe,
    param_grid=param_grid_Cat,
    scoring='f1',
    n_jobs=5,
    cv=5,
    error_score='raise'
)

gs_Cat.fit(features_train, targets_train)

gs_Cat_best_score = gs_Cat.best_score_
gs_Cat_best_params = gs_Cat.best_params_

# лучшее значение f1 на кросс-валидации
print(f'best_score: {gs_Cat_best_score}')
# лучшие гиперпараметры
print(f'best_params: {gs_Cat_best_params}')

Долго учится, я устал ждать

#### аргумент text_features

In [23]:
features_train_df = pd.DataFrame({'lemma_text':features_train.values}, index=features_train.index)
model_Cat = CatBoostClassifier(random_state=RANDOM_STATE, allow_writing_files=False, verbose=False, auto_class_weights='Balanced', text_features=['lemma_text',])
print('CV score', cross_val_score(
    model_Cat,
    features_train_df,
    targets_train,
    scoring='f1',
    n_jobs=-1,
    error_score='raise'
    )
)

CV score [0.72796329 0.72073499 0.71446822 0.72850024 0.73560252]


In [24]:
print(np.mean([0.72796329, 0.72073499, 0.71446822, 0.72850024, 0.73560252]))

0.725453852


#### Pipeline с tf-idf

In [25]:
model_Cat = CatBoostClassifier(random_state=RANDOM_STATE, allow_writing_files=False, verbose=False, auto_class_weights='Balanced')
tfidf_cat_pipe = Pipeline([
    ('tvec', vectorizer),
    ('catboost', model_Cat)
])
cv_score = cross_val_score(
    tfidf_cat_pipe,
    features_train,
    targets_train,
    scoring='f1',
    n_jobs=-1
    )
print('CV score', cv_score)
print('mean', np.mean(cv_score))

CV score [0.76008418 0.7578388  0.74862905 0.76533287 0.76337778]
mean 0.7590525336124555


### RandomForestClassifier

In [26]:
%%time
model_forest = RandomForestClassifier(random_state=RANDOM_STATE, class_weight="balanced")
tfidf_forest_pipe = Pipeline([
    ('tvec', vectorizer),
    ('forest', model_forest)
])
cv_score = cross_val_score(
    tfidf_forest_pipe,
    features_train,
    targets_train,
    scoring='f1',
    n_jobs=-1
    )
print('CV score', cv_score)
print('mean', np.mean(cv_score))

CV score [0.66082603 0.64920675 0.63640974 0.65449087 0.64448959]
mean 0.6490845949107026
CPU times: total: 703 ms
Wall time: 13min 50s


### KNeighborsClassifier

In [27]:
%%time
model_knn = KNeighborsClassifier()
tfidf_knn_pipe = Pipeline([
    ('tvec', vectorizer),
    ('knn', model_knn)
])
cv_score = cross_val_score(
    tfidf_knn_pipe,
    features_train,
    targets_train,
    scoring='f1',
    n_jobs=-1
    )
print('CV score', cv_score)
print('mean', np.mean(cv_score))

CV score [0.32354778 0.28598073 0.29800499 0.30443038 0.3037575 ]
mean 0.30314427530204646
CPU times: total: 734 ms
Wall time: 1min 54s


Скрее всего  надо было делать балансировку через downsample или upsample

### LogisticRegression

In [30]:
%%time
model_log = LogisticRegression(random_state=RANDOM_STATE, class_weight='balanced')


param_grid_log = {'log__solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
                 'log__penalty': ['l1', 'l2', 'elasticnet', 'none']}

tfidf_log_pipe = Pipeline([
    ('tvec', vectorizer),
    ('log', model_log)
])

gs_log = GridSearchCV(
    tfidf_log_pipe,
    param_grid=param_grid_log,
    scoring='f1',
    n_jobs=-1,
    cv=5,
)

gs_log.fit(features_train, targets_train)

gs_log_best_score = gs_log.best_score_
gs_log_best_params = gs_log.best_params_

# лучшее значение f1 на кросс-валидации
print(f'best_score: {gs_log_best_score}')
# лучшие гиперпараметры
print(f'best_params: {gs_log_best_params}')

C:\ProgramData\Anaconda3\envs\practicum\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.74855796        nan        nan        nan 0.66226117
 0.75050772 0.75052534 0.75055064        nan 0.75057697 0.7515628
        nan        nan        nan        nan        nan        nan
 0.72668521        nan 0.57624604        nan 0.72523045 0.73466493]
  warnings.warn(


best_score: 0.7515628038076635
best_params: {'log__penalty': 'l2', 'log__solver': 'saga'}
CPU times: total: 43 s
Wall time: 8min 11s


C:\ProgramData\Anaconda3\envs\practicum\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


## Выводы

CatBoost показал лучшие результаты по метрике f1, используем его на тестовой выборке

In [29]:
%%time
model_Cat = CatBoostClassifier(random_state=RANDOM_STATE, allow_writing_files=False, verbose=False, auto_class_weights='Balanced')
tfidf_cat_pipe = Pipeline([
    ('tvec', vectorizer),
    ('catboost', model_Cat)
])
tfidf_cat_pipe.fit(features_train, targets_train)
predictions = tfidf_cat_pipe.predict(features_test)
print('f1 score на тестовой выборке:', f1_score(targets_test, predictions))

f1 score на тестовой выборке: 0.7613540197461213
CPU times: total: 1h 14min 34s
Wall time: 10min 43s


### Данные:
Была произведена **лемматизация** текста.  
Признаки закодированы с помощью **Tf-Idf**.  
### Модели:
Проанализированы 4 модели.  
На кросс валидации лучшим себя показал CatBoost.  
Его метрика F1 score на тестовой выборке **0.7614**